In [13]:
import import_ipynb
import numpy as np
from numpy import genfromtxt
import pandas as pd
import rtree
from rtree import index

In [7]:
def lower_upper_check(kdnodes):
    for i in range(len(kdnodes)):
        for j in range(len(kdnodes[i][0])):
            if kdnodes[i][0][j][0] > kdnodes[i][0][j][1]:
                print('found invalid kdnode',i,kdnodes[i])
                break

In [8]:
def load_partitions_from_file(path):
    '''
    the loaded stretched_kdnodes: [num_dims, l1,l2,...,ln, u1,u2,...,un, size, id, pid, left_child,id, right_child_id]
    '''
    stretched_kdnodes = genfromtxt(path, delimiter=',')
    num_dims = int(stretched_kdnodes[0,0])
    kdnodes = []
    
    for i in range(len(stretched_kdnodes)):
        domains = [ [stretched_kdnodes[i,k+1],stretched_kdnodes[i,1+num_dims+k]] for k in range(num_dims) ]
        row = [domains]
        row.append(stretched_kdnodes[i,2*num_dims+1])
        
        # to be compatible with qd-tree's partition, that do not have the last 4 attributes
        if len(stretched_kdnodes[i]) > 2*num_dims+2:
            row.append(stretched_kdnodes[i,-4])
            row.append(stretched_kdnodes[i,-3])
            row.append(stretched_kdnodes[i,-2])
            row.append(stretched_kdnodes[i,-1])
    
        kdnodes.append(row)
    
    return kdnodes

In [32]:
# from Utils import *
# path = 'C:/Users/Cloud/iCloudDrive/HUAWEI_LKD/HDFSExperiment/generated_partitions/nora_partitions'
# path = 'C:/Users/Cloud/iCloudDrive/HUAWEI_LKD/HDFSExperiment/generated_partitions/qd_tree_partitions'

# path = '/home/cloudray/NORA_Partitions/nora_partitions'
path = '/home/cloudray/NORA_Partitions/qd_tree_partitions'

saved_partitions = load_partitions_from_file(path)
lower_upper_check(saved_partitions)

In [25]:
raw_data_path = '/home/cloudray/Downloads/TPCH_12M_8Field.csv'

In [ ]:
# dataset = genfromtxt(raw_data_path, delimiter=',')

In [ ]:
# # check if every point in dataset can find a partition that holds it
# for i in range(len(dataset)):
#     if i% 100 == 0:
#         print('processing..',i)
#     find_tag = False
#     for j in range(saved_partitions):
#         if dataset[i][1] >= saved_partitions[j][0][0][0] and dataset[i][1] <= saved_partitions[j][0][0][1]\
#         and dataset[i][2] >= saved_partitions[j][0][1][0] and dataset[i][2] <= saved_partitions[j][0][1][1]:
#             find_tag = True
#             break
#     if find_tag == False:
#         print('no partition that can hold the',i,'point: ',dataset[i])

In [19]:
# chunk_count = 0
# for chunk in pd.read_csv(raw_data_path, chunksize=10000):
#     print('current chunk: ', chunk_count)
#     dataset = chunk.to_numpy()
#     for i in range(len(dataset)):
#         find_tag = False
#         for j in range(len(saved_partitions)):
#             if dataset[i][1] >= saved_partitions[j][0][0][0] and dataset[i][1] <= saved_partitions[j][0][0][1]\
#             and dataset[i][2] >= saved_partitions[j][0][1][0] and dataset[i][2] <= saved_partitions[j][0][1][1]:
#                 find_tag = True
#                 break
#         if find_tag == False:
#             print('no partition that can hold the',i,'point: ',dataset[i])
#             break
#     chunk_count += 1

In [26]:
def kdnode_2_border(kdnode):
    lower = [domain[0] for domain in kdnode[0]]
    upper = [domain[1] for domain in kdnode[0]]
    border = tuple(lower + upper) # non interleave
    return border

In [33]:
# test after indexing partitions by Rtree, is there any record that cannot find a partition to hold?

p = index.Property()
p.leaf_capacity = 100 # cannot be less than 100, indicate the maximum capacity
p.fill_factor = 0.5
p.overwrite = True

partition_index = index.Index(properties = p)
for i in range(len(saved_partitions)):
    #partition_index.insert(int(partitions[i][-4]), kdnode_2_border(partitions[i]))
    partition_index.insert(i, kdnode_2_border(saved_partitions[i]))

In [34]:
print(partition_index)
print(partition_index.get_size())
print(len(saved_partitions))

rtree.index.Index(bounds=[1.0, 1.0, 400000.0, 20000.0], size=53)
53


In [39]:
overlap_pids = list(partition_index.intersection((275983.0, 11023.0, 275983.0, 11023.0)))
print(overlap_pids)
print(partition_index.leaves())
print(saved_partitions[7])

[7]
[(0, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52], [1.0, 1.0, 400000.0, 20000.0])]
[[[240110.55543950442, 278102.008168012], [10252.013667268735, 14843.77380863712]], 263925.0]


In [31]:
import copy
idx_copy = copy.deepcopy(partition_index)

In [30]:
chunk_count = 0
for chunk in pd.read_csv(raw_data_path, chunksize=100000):
    print('current chunk: ', chunk_count)
    dataset = chunk.to_numpy()
    for i in range(len(dataset)):
        point_border = (dataset[i,1], dataset[i,2], dataset[i,1], dataset[i,2])
        overlap_pids = list(partition_index.intersection(point_border))
        if len(overlap_pids) == 0:
            print('no partition that can hold the',i,'point: ',dataset[i])
            break
    chunk_count += 1

current chunk:  0
current chunk:  1
current chunk:  2
current chunk:  3
current chunk:  4
current chunk:  5
current chunk:  6
current chunk:  7
current chunk:  8
current chunk:  9
current chunk:  10
current chunk:  11
current chunk:  12
current chunk:  13
current chunk:  14
current chunk:  15
current chunk:  16
current chunk:  17
current chunk:  18
current chunk:  19
current chunk:  20
current chunk:  21
current chunk:  22
current chunk:  23
current chunk:  24
current chunk:  25
current chunk:  26
current chunk:  27
current chunk:  28
current chunk:  29
current chunk:  30
current chunk:  31
current chunk:  32
current chunk:  33
current chunk:  34
current chunk:  35
current chunk:  36
current chunk:  37
current chunk:  38
current chunk:  39
current chunk:  40
current chunk:  41
current chunk:  42
current chunk:  43
current chunk:  44
current chunk:  45
current chunk:  46
current chunk:  47
current chunk:  48
current chunk:  49
current chunk:  50
current chunk:  51
current chunk:  52
cur